# Hotel Bookings

In [6]:
import pandas as pd

In [7]:
hotels = pd.read_csv('bookings.csv', sep=';')

Проверим данные датасета

In [8]:
hotels

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017-08-30,2017,August,35,30,2,5,...,2,0.0,0,BB,BEL,A,A,Transient,Check-Out,2017-09-06
119386,City Hotel,0,102,2017-08-31,2017,August,35,31,2,5,...,3,0.0,0,BB,FRA,E,E,Transient,Check-Out,2017-09-07
119387,City Hotel,0,34,2017-08-31,2017,August,35,31,2,5,...,2,0.0,0,BB,DEU,D,D,Transient,Check-Out,2017-09-07
119388,City Hotel,0,109,2017-08-31,2017,August,35,31,2,5,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2017-09-07


Посмотрим на типы данных

In [9]:
hotels.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

Приведем названия колонок к нижнему регистру и заменим пробелы на знак нижнего подчеркивания

In [13]:
def underscore_rename(name):
    return name.replace(' ', '_').lower()


In [14]:
hotels = hotels.rename(columns=underscore_rename)


Посмотрим на все колонки датасета 

In [17]:
hotels.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

 Выведем топ-5 стран, жители которых совершили наибольшее число успешных бронирований 

In [19]:
hotels.query('is_canceled == 0').country.value_counts()[:5]

PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: country, dtype: int64

In [20]:
hotels.query('is_canceled == 0').groupby('country', as_index=False).agg({'is_canceled':'count'}).sort_values('is_canceled', ascending = False)[:5]

,country,is_canceled
125,PRT,21071
57,GBR,9676
54,FRA,8481
50,ESP,6391
42,DEU,6069


Выясним на сколько ночей в среднем бронируют отели разных типов

In [21]:
hotels.groupby('hotel', as_index = False).agg({'stays_total_nights':'mean'}).round(2)

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type).
Такое может произойти, например, по причине овербукинга. Посмотрим, сколько подобных наблюдений встретилось в датасете

In [22]:
hotels[['reserved_room_type', 'assigned_room_type']]

,reserved_room_type,assigned_room_type
0,C,C
1,C,C
2,A,C
3,A,A
4,A,A
...,...,...
119385,A,A
119386,E,E
119387,D,D
119388,A,A


In [24]:
hotels.query('reserved_room_type != assigned_room_type').shape[0]

14917

Проанализируем даты запланированного прибытия. 
На какой месяц чаще всего успешно оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?

In [33]:
hotels.query('arrival_date_year == 2016 and is_canceled == 0').arrival_date_month.value_counts()

October      3689
May          3563
September    3372
April        3367
March        3347
August       3238
June         3196
July         3073
November     2818
February     2554
December     2462
January      1691
Name: arrival_date_month, dtype: int64

In [32]:
hotels.query('arrival_date_year == 2017 and is_canceled == 0').arrival_date_month.value_counts()

May         3551
July        3329
March       3298
June        3208
April       3198
August      3109
February    2818
January     2431
Name: arrival_date_month, dtype: int64



Сгруппируем данные по годам и проверим, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего 
в каждый из периодов

In [44]:
hotels.query('hotel == "City Hotel" and is_canceled == 1') \
      .groupby('arrival_date_year') \
      .agg({'arrival_date_month': 'value_counts'})

arrival_date_month
arrival_date_year arrival_date_month                    
2015              September                         1543
                  October                           1321
                  August                            1232
                  July                               939
                  December                           668
                  November                           301
2016              October                           1947
                  June                              1720
                  September                         1567
                  April                             1539
                  May                               1436
                  November                          1360
                  August                            1247
                  March                             1108
                  December                          1072
                  July                              1043
                  February                           930
                  January                            438
2017              May                               2217
                  April                             1926
                  June                              1808
                  July                              1324
                  March                             1278
                  August                            1123
                  January                           1044
                  February                           971

In [46]:
#Второй вариант кода
(
    hotels
        .query('hotel == "City Hotel" and is_canceled == 1')
        .groupby('arrival_date_year')
        .arrival_date_month
        .value_counts()
)


arrival_date_year  arrival_date_month
2015               September             1543
                   October               1321
                   August                1232
                   July                   939
                   December               668
                   November               301
2016               October               1947
                   June                  1720
                   September             1567
                   April                 1539
                   May                   1436
                   November              1360
                   August                1247
                   March                 1108
                   December              1072
                   July                  1043
                   February               930
                   January                438
2017               May                   2217
                   April                 1926
                   June                  1

Посмотрим на числовые характеристики трёх переменных: adults, children и babies. 
Какая из них имеет наибольшее среднее значение?

In [51]:
hotels[['adults', 'children', 'babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

In [52]:
hotels[['adults', 'children', 'babies']].describe()

,adults,children,babies
count,119390.000000,119386.000000,119390.000000
mean,1.856403,0.103890,0.007949
std,0.579261,0.398561,0.097436
min,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000
50%,2.000000,0.000000,0.000000
75%,2.000000,0.000000,0.000000
max,55.000000,10.000000,10.000000


Создадим колонку total_kids, объединив children и babies. 
Выясним, отели какого типа в среднем пользуются большей популярностью у клиентов с детьми

In [55]:
hotels['total_kids'] = hotels['children'] + hotels['babies']

In [64]:
hotels.total_kids.sort_values(ascending=False)

328       10.0
46619     10.0
78656      9.0
19718      3.0
107837     3.0
          ... 
119389     0.0
40600      NaN
40667      NaN
40679      NaN
41160      NaN
Name: total_kids, Length: 119390, dtype: float64

In [75]:
hotels.groupby('hotel') \
      .agg({'total_kids':'mean'})\
      .round(2)

,total_kids
hotel,
City Hotel,0.10
Resort Hotel,0.14


In [73]:
hotels.groupby('hotel') \
      .agg({'total_kids':'mean'})\
      .round(2)\
      .max()

total_kids    0.14
dtype: float64

 Создадим переменную has_kids, которая принимает значение True, 
 если клиент при бронировании указал хотя бы одного ребенка (total_kids),
 в противном случае – False. 
 Посчитаем отношение количества ушедших пользователей к общему количеству клиентов, 
 выраженное в процентах (churn rate). 
 Выясним среди какой группы показатель выше

In [77]:
hotels['has_kids'] = hotels.total_kids > 0 

In [78]:
hotels

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date,total_kids,has_kids
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0,False
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0,False
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02,0.0,False
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02,0.0,False
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017-08-30,2017,August,35,30,2,5,...,0,BB,BEL,A,A,Transient,Check-Out,2017-09-06,0.0,False
119386,City Hotel,0,102,2017-08-31,2017,August,35,31,2,5,...,0,BB,FRA,E,E,Transient,Check-Out,2017-09-07,0.0,False
119387,City Hotel,0,34,2017-08-31,2017,August,35,31,2,5,...,0,BB,DEU,D,D,Transient,Check-Out,2017-09-07,0.0,False
119388,City Hotel,0,109,2017-08-31,2017,August,35,31,2,5,...,0,BB,GBR,A,A,Transient,Check-Out,2017-09-07,0.0,False


In [84]:
#Жоля пользователей у которых не было детей и отменили бронь
hotels.query('has_kids == False and is_canceled == 1').shape[0] / hotels.query('has_kids == False').shape[0] 

0.37221283323338605

In [85]:
#Доля пользователей у которых были дети и отменили бронь
hotels.query('has_kids == True and is_canceled == 1').shape[0] / hotels.query('has_kids == True').shape[0] 

0.3492284612087441